# Duet

Learning to Match Using Local and Distributed Representations of Text for Web Search

https://arxiv.org/abs/1610.08136

In [1]:
from __future__ import print_function
import numpy as np
import cntk as C
from cntk.learner import sgd, learning_rate_schedule, UnitType
from cntk.utils import ProgressPrinter
from cntk.layers import Dense
from cntk.models import Sequential

ImportError: No module named 'cntk'